In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime,timedelta
from pybaseball import batting_stats_range,pitching_stats_range,playerid_reverse_lookup
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen as Ureq
warnings.filterwarnings("ignore")

In [11]:
bat_df = pd.read_csv("Clean_Bat.csv",index_col = "Unnamed: 0")
pitch_df = pd.read_csv("Clean_Pitch.csv",index_col = "Unnamed: 0")
pitch_df.tail()

Name        Date         Opp Team  Home_Away  GS   IP  H  R  \
118    Brendan White  2023-07-07     Toronto  DET        0.0   0  1.0  0  0   
119      Mitch White  2023-07-07     Detroit  TOR        1.0   0  1.2  2  1   
120   Devin Williams  2023-07-07  Cincinnati  MIL        0.0   0  0.2  0  0   
121  Trevor Williams  2023-07-07       Texas  WSN        0.0   1  6.0  7  4   
122       Alex Young  2023-07-07   Milwaukee  CIN        1.0   0  0.2  1  0   

     ER  ...  IBB  GDP  SF  SB  CS  PO  BF  GB/FB    LD   PU  
118   0  ...    0    1   0   0   0   0   3   0.67  0.33  0.0  
119   1  ...    0    1   0   0   0   0   7   0.40  0.20  0.2  
120   0  ...    0    0   0   0   0   0   3    NaN   NaN  NaN  
121   4  ...    0    0   0   0   0   0  28   0.40  0.40  0.0  
122   0  ...    0    0   0   0   0   0   4   0.50  0.50  0.0  

[5 rows x 27 columns]

In [12]:
start_date = bat_df.iloc[-1].Date
now = datetime.now()
end_date = now.strftime("%Y-%m-%d")
start_date = datetime.strptime(start_date,"%Y-%m-%d")
total_bat_df = pd.DataFrame()
total_pitch_df = pd.DataFrame()
while start_date < now:
    next_day = start_date + timedelta(days=1)
    if next_day > now:
        break
    temp_bat_df = batting_stats_range(start_date.strftime("%Y-%m-%d"),)
    temp_pitch_df = pitching_stats_range(start_date.strftime("%Y-%m-%d"),)
    start_date = next_day
    data = playerid_reverse_lookup(temp_bat_df['mlbID'].unique(),key_type = "mlbam")
    last_list = []
    first_list = []
    for x in range(len(data)):
        last_list.append(data.iloc[x]['name_last'].capitalize())
        first_list.append(data.iloc[x]['name_first'].capitalize())
    data['name_last'] = last_list
    data['name_first'] = first_list
    data['Name'] = data['name_first'] + " " + data['name_last']
    key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
    for name in temp_bat_df['mlbID']:
        try:
            key_dict[name]
        except:
            key_dict[name] = temp_bat_df[temp_bat_df['mlbID'] == name]['Name']
    temp_bat_df['mlbID'] = temp_bat_df['mlbID'].apply(lambda x: key_dict[x])
    temp_bat_df['Name'] = temp_bat_df['mlbID']

    names = temp_pitch_df['mlbID'].unique()
    int_names = [int(name) for name in names]
    data = playerid_reverse_lookup(int_names,key_type = "mlbam")
    last_list = []
    first_list = []
    for x in range(len(data)):
        last_list.append(data.iloc[x]['name_last'].capitalize())
        first_list.append(data.iloc[x]['name_first'].capitalize())
    data['name_last'] = last_list
    data['name_first'] = first_list
    data['Name'] = data['name_first'] + " " + data['name_last']
    data['key_mlbam'] = [str(name) for name in data['key_mlbam']]
    key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
    for name in temp_pitch_df['mlbID']:
        try:
            key_dict[name]
        except:
            key_dict[name] = temp_pitch_df[temp_pitch_df['mlbID'] == name]['Name']
    temp_pitch_df['mlbID'] = temp_pitch_df['mlbID'].apply(lambda x: key_dict[x])
    temp_pitch_df['Name'] = temp_pitch_df['mlbID']
    temp_bat_df['Team'] = temp_bat_df.Lev + temp_bat_df.Tm
    temp_pitch_df['Team'] = temp_pitch_df.Lev + temp_pitch_df.Tm
    team_dict = {'Maj-NLMilwaukee':"MIL",
                 'Maj-ALLos Angeles': "LAA",
                 'Maj-NLAtlanta': "ATL",
                 'Maj-NLArizona': "ARI",
                 'Maj-NLNew York': "NYN",
                 'Maj-ALHouston': "HOU",
                 'Maj-NLCincinnati':"CIN",
                 'Maj-NLSt. Louis': "SLN",
                 'Maj-NLSan Diego': "SDN",
                 'Maj-NLWashington': "WSN",
                 'Maj-ALKansas City': "KCR",
                 'Maj-ALCleveland': "CLE",
                 'Maj-NLPittsburgh':"PIT",
                 'Maj-NLChicago':"CHN", 
                 'Maj-ALChicago':"CHA",
                 'Maj-NLMiami':"MIA",
                 'Maj-ALOakland':"OAK",
                 'Maj-ALBoston':"BOS",
                 'Maj-ALTampa Bay':"TBA",
                 'Maj-ALMinnesota':"MIN",
                 'Maj-ALDetroit':"DET",
                 'Maj-NLSan Francisco':"SFN",
                 'Maj-NLLos Angeles':"LAN",
                 'Maj-ALToronto':"TOR",
                 'Maj-NLColorado':"COL",
                 'Maj-ALTexas':"TEX",
                 'Maj-NLPhiladelphia':"PHI",
                 'Maj-ALBaltimore':"BAL",
                 'Maj-ALSeattle':"SEA",
                 'Maj-ALNew York':"NYA"}
    temp_bat_df['Team'] = temp_bat_df['Team'].apply(lambda x: team_dict[x])
    temp_pitch_df['Team'] = temp_pitch_df['Team'].apply(lambda x: team_dict[x])
    temp_bat_df.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
    temp_pitch_df.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
    total_bat_df = pd.concat([total_bat_df,temp_bat_df])
    total_pitch_df = pd.concat([total_pitch_df,temp_pitch_df])
total_bat_df['Home_Away'] = total_bat_df[total_bat_df.columns[4]]
total_pitch_df['Home_Away'] = total_pitch_df[total_pitch_df.columns[4]]
bat_important_stats = ['Name','Date','Opp','Team','Home_Away','PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS']
pitch_important_stats = ['Name','Date','Opp','Team','Home_Away','GS','IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']
total_bat_df = total_bat_df[bat_important_stats]
total_pitch_df = total_pitch_df[pitch_important_stats]
bat_df = pd.concat([bat_df,total_bat_df])
pitch_df = pd.concat([pitch_df,total_pitch_df])
bat_df['Home_Away'] = bat_df['Home_Away'].replace('@',1)
bat_df['Home_Away'] = bat_df['Home_Away'].replace('', 0) 
bat_df['Date'] = pd.to_datetime(bat_df['Date'])
pitch_df['Home_Away'] = pitch_df['Home_Away'].replace('@',1)
pitch_df['Home_Away'] = pitch_df['Home_Away'].replace('', 0) 
pitch_df['Date'] = pd.to_datetime(pitch_df['Date'])
pitch_df.to_csv("Clean_Pitch.csv")
bat_df.to_csv("Clean_Bat.csv")

In [13]:
team_dict = {'Milwaukee Brewers':"MIL",
             'Los Angeles Angels': "LAA",
             'Atlanta Braves': "ATL",
             "Arizona D'Backs": "ARI",
             'New York Mets': "NYN",
             'Houston Astros': "HOU",
             'Cincinnati Reds':"CIN",
             'St. Louis Cardinals': "SLN",
             'San Diego Padres': "SDN",
             'Washington Nationals': "WSN",
             'Kansas City Royals': "KCR",
             'Cleveland Guardians': "CLE",
             'Pittsburgh Pirates':"PIT",
             'Chicago Cubs':"CHN", 
             'Chicago White Sox':"CHA",
             'Miami Marlins':"MIA",
             'Oakland Athletics':"OAK",
             'Boston Red Sox':"BOS",
             'Tampa Bay Rays':"TBA",
             'Minnesota Twins':"MIN",
             'Detroit Tigers':"DET",
             'San Francisco Giants':"SFN",
             'Los Angeles Dodgers':"LAN",
             'Toronto Blue Jays':"TOR",
             'Colorado Rockies':"COL",
             'Texas Rangers':"TEX",
             'Philadelphia Phillies':"PHI",
             'Baltimore Orioles':"BAL",
             'Seattle Mariners':"SEA",
             'New York Yankees':"NYA"}
url = 'https://www.baseball-reference.com/leagues/MLB-schedule.shtml'
uClient = Ureq(url)
raw_content = uClient.read()
page_soup = soup(raw_content, "html.parser")
html = list(page_soup.children)[3]
games = str(html).split('h3')[3:]
end_date = datetime.today()
total_df = pd.read_csv("Previous_Games.csv",index_col = "Unnamed: 0")
this_year = total_df[total_df['Date'] > "2023-01-01"]
y = len(this_year.Date.unique())*2
try:
    date = datetime.strptime(games[y].split('>')[1].split('<')[0].split(', ')[1] + " " + games[0].split('>')[1].split('<')[0].split(', ')[2],"%B %d %Y")
except:
    print("Currently have all games played")
    date = end_date
while date < end_date:
    date = datetime.strptime(games[y].split('>')[1].split('<')[0].split(', ')[1] + " " + games[0].split('>')[1].split('<')[0].split(', ')[2],"%B %d %Y")
    y += 1
    x = 1
    while x < (len(games[y].split('.shtml">'))-3):
        temp_list = []
        temp_list.append(games[y].split('.shtml">')[x].split('<')[0])
        temp_list.append(games[y].split('.shtml">')[x].split('<')[1].split('(')[1].split(')')[0])
        temp_list.append(games[y].split('.shtml">')[x+1].split('<')[0])
        temp_list.append(games[y].split('.shtml">')[x+1].split('<')[1].split('(')[1].split(')')[0])
        temp_df = pd.DataFrame(temp_list).T
        new_date = date.strftime("%Y-%m-%d")
        temp_df['Date'] = new_date
        temp_df.columns = ['Visitor','V_Score','Home','H_Score','Date']
        temp_df['Visitor'] = temp_df['Visitor'].apply(lambda x: team_dict[x])
        temp_df['Home'] = temp_df['Home'].apply(lambda x: team_dict[x])
        total_df = pd.concat([total_df,temp_df])
        x+=3
    y += 1
    try:
        date = datetime.strptime(games[y].split('>')[1].split('<')[0].split(', ')[1] + " " + games[0].split('>')[1].split('<')[0].split(', ')[2],"%B %d %Y")
    except:
        date = end_date
total_df.to_csv("Previous_Games.csv")
total_df.tail()

Currently have all games played


Visitor  V_Score Home  H_Score        Date
0     BAL        6  MIN        2  2023-07-08
0     CHN        3  NYA        6  2023-07-08
0     NYN        1  SDN        3  2023-07-08
0     COL        3  SFN        5  2023-07-08
0     ATL        6  TBA        1  2023-07-08

In [15]:
## It's updated at least by 2:50